In [0]:
%tensorflow_version 2.x

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model

In [7]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2020-04-22 09:36:44--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2020-04-22 09:36:44--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdfa75375fb5e4e2d90cefe6b99.dl.dropboxusercontent.com/zip_download_get/AZvjh8guopEpDsnUwuLt65BVISMGxL9y5ttf9JIREwnjM9ujNj3NeRcCu4Gk1yfc34HyD6aPc7HWNmg-QSEh7p6QRK8hTniWh4VayXUY31_A9A [following]
--2020-04-22 09:36:45--  https://ucdfa75375fb5e4e2d90cefe6b99.dl.dropboxusercontent.com/zip_download_get/AZvjh8guopEpDsnUwuLt65BVISMGxL9y5ttf9JIREwnjM9ujNj3NeRcCu4Gk1yfc34HyD6aPc7HWNmg-Q

In [8]:
!unzip AAAw1_JIzpuVvwteJCma0xMla?dl=0

Archive:  AAAw1_JIzpuVvwteJCma0xMla?dl=0
mapname:  conversion of  failed
 extracting: train.csv               

1 archive had fatal errors.


In [0]:
file='train.csv'

In [0]:
ques=pd.read_csv(file)

In [11]:
ques.tail(100)

,qid,question_text,target
1306022,fffb42806a68603771e1,Does Jordan Peterson go to church?,0
1306023,fffb433c9a3c4d91c80f,If you had an idea to create a very cheap hidd...,0
1306024,fffb60597913215dfe58,When do one actually need to take a pregnancy ...,0
1306025,fffb79a4422e61289508,"Is it just me, or was Prince the singer ugly a...",1
1306026,fffb7af82c269dd9e46b,Could we finally give Einstein's first wife cr...,1
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [12]:
ques.drop(['qid'],axis=1)

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0
...,...,...
1306117,What other technical skills do you need as a c...,0
1306118,Does MS in ECE have good job prospects in USA ...,0
1306119,Is foam insulation toxic?,0
1306120,How can one start a research project based on ...,0


In [16]:
ques['target'].value_counts()

0    1225312
1      80810
Name: target, dtype: int64

In [13]:
ques.shape

(1306122, 3)

In [0]:
ques_train,ques_test=train_test_split(ques,test_size=0.2,random_state=2)

In [0]:
x_train=ques_train['question_text']
y_train=ques_train['target']
x_test=ques_test['question_text']
y_test=ques_test['target']

In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [24]:
sent_lens=[]
for sent in ques_train['question_text']:
    sent_lens.append(len(word_tokenize(sent)))
   
    

Streaming output truncated to the last 5000 lines.
['I', '’', 'm', 'taking', 'a', 'trip', 'to', 'the', 'US', 'this', 'summer', 'and', 'my', 'bank', 'card', 'won', '’', 't', 'work', '.', 'My', 'mom', 'is', 'giving', 'me', 'hers', 'to', 'use', '.', 'Is', 'it', 'illegal', 'to', 'use', 'hers', 'when', 'I', 'have', 'her', 'permission', '?', 'We', 'also', 'have', 'different', 'last', 'names', '.']
['How', 'effectively', 'are', 'animal', 'welfare', 'organizations', 'in', 'the', 'U.S.', 'using', 'social', 'media', '?', 'What', 'studies', 'or', 'research', 'exists', 'discussing', 'this', 'topic', '?']
['How', 'should', 'we', 'start', 'getting', 'tender', 'in', 'India', ',', 'as', 'a', 'fresher', 'in', 'the', 'field', ',', 'what', 'all', 'things', 'should', 'we', 'consider', 'working', 'on', '?']
['Can', 'I', 'keep', 'chipmunks', 'and/or', 'squirrels', 'as', 'pets', '?']
['Is', 'Donald', 'Trump', 'Putin', "'s", 'Patsy', '?']
['What', 'inspired', 'George', 'Mann', 'to', 'write', 'the', 'book', '`

In [25]:
max(sent_lens)

412

In [31]:
np.quantile(sent_lens,0.98)

39.0

In [0]:
max_len = 39

tok = Tokenizer(char_level=False,split=' ')

tok.fit_on_texts(x_train)

In [0]:
sequences_train = tok.texts_to_sequences(x_train)

In [0]:
vocab_len=len(tok.index_word.keys())

In [0]:
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len)

In [21]:
vocab_len

196201

In [2]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2020-04-22 09:20:33--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2020-04-22 09:20:33--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2020-04-22 09:20:34--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [3]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [20]:
!tail -10 glove.42B.300d.txt

sigarms 0.18917 -0.3181 -0.43749 0.61209 -0.031357 0.1568 0.30505 -0.39915 0.30101 -0.17353 -0.032278 -0.29338 -0.16598 -0.15038 -0.29364 -0.062909 -0.32706 0.13117 0.010177 0.078467 -0.32796 0.008198 -0.34072 0.1286 -0.27844 0.41588 -0.045799 0.45221 -0.065252 -0.055483 0.50665 -0.067356 -0.29428 -0.12657 -0.024669 -0.37284 0.23507 -0.48212 0.041255 -0.081911 0.59964 0.2047 -0.23401 -0.34091 -0.13306 0.36 -0.00697 0.6013 0.50416 0.099127 -0.031218 0.40532 -0.16901 -0.4664 0.14094 0.74539 0.15076 0.18017 -0.51756 -0.59651 -0.20991 -0.5506 -0.023214 -0.4166 -0.33079 -0.048234 -0.16065 0.36674 0.0072976 0.051298 -0.38274 0.57222 -0.34032 0.16816 -0.40298 -0.12421 0.16196 -0.15993 0.43126 -0.4145 -0.48541 -0.28735 0.34193 -0.029869 0.70822 0.01381 -0.034878 -0.26084 0.20595 0.64408 -0.056202 -0.1277 0.28507 -0.16824 -0.018927 -0.24652 -0.10403 0.32856 -0.43073 -0.10308
katuna -0.21887 -0.30785 -0.28557 0.36186 -0.20892 -0.011439 -0.39597 0.27104 0.51042 -0.29147 -0.18927 0.167 -0.043189 -

In [0]:
def RNN(embedding_matrix,embed_dim,max_len,vocab_size):
    inputs = Input(name='inputs',shape=[max_len])
    
    layer = Embedding(vocab_size+1,embed_dim,input_length=max_len,
                      mask_zero=True,weights=[embedding_matrix],trainable=False)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model



In [22]:
embedding_index={}
f = open('/content/glove.42B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embedding_index))

Found 1917494 word vectors.


In [0]:
embedding_matrix = np.zeros((vocab_len+1,300))
words_not_available=0
for word, i in tok.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
      words_not_available+=1

In [24]:
print(words_not_available)

62201


In [26]:
model = RNN(embedding_matrix,300,39,vocab_len)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 39)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 39, 300)           58860600  
_________________________________________________________________
lstm (LSTM)                  (None, 64)                93440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257   

In [0]:
model.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'])

In [0]:
sequences_test = tok.texts_to_sequences(x_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test,
                                               maxlen=max_len)

In [29]:
model.fit(sequences_matrix_train,y_train.values,batch_size=5000,
          epochs=7,validation_data=(sequences_matrix_test,y_test.values))

Epoch 1/7
209/209 [==============================] - 573s 3s/step - loss: 0.1636 - accuracy: 0.9449 - val_loss: 0.1188 - val_accuracy: 0.9527
Epoch 2/7
209/209 [==============================] - 566s 3s/step - loss: 0.1167 - accuracy: 0.9540 - val_loss: 0.1118 - val_accuracy: 0.9555
Epoch 3/7
209/209 [==============================] - 566s 3s/step - loss: 0.1108 - accuracy: 0.9563 - val_loss: 0.1080 - val_accuracy: 0.9570
Epoch 4/7
209/209 [==============================] - 567s 3s/step - loss: 0.1065 - accuracy: 0.9579 - val_loss: 0.1073 - val_accuracy: 0.9573
Epoch 5/7
209/209 [==============================] - 564s 3s/step - loss: 0.1039 - accuracy: 0.9588 - val_loss: 0.1071 - val_accuracy: 0.9567
Epoch 6/7
209/209 [==============================] - 566s 3s/step - loss: 0.1015 - accuracy: 0.9596 - val_loss: 0.1041 - val_accuracy: 0.9585
Epoch 7/7
209/209 [==============================] - 562s 3s/step - loss: 0.0996 - accuracy: 0.9604 - val_loss: 0.1029 - val_accuracy: 0.9589


In [0]:
predictions=model.predict(sequences_matrix_test)

In [0]:
from sklearn.metrics import roc_auc_score

In [32]:
roc_auc_score(y_test,predictions)

0.962328359392454